<a href="https://colab.research.google.com/github/ChemRacer/molecular_representation_examples/blob/main/Chapter_4/CMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Molecular_representations/molecular_representation_examples/Chapter_4

In [ ]:
!pip install seaborn pandas matplotlib dscribe ase
path_to_structure='/content/drive/MyDrive/Colab Notebooks/Molecular_representations/molecular_representation_examples/example_structures/glycidol.xyz'

In [ ]:
# An example of how to generate a Coulomb matrices (CM)
# for glycidol using DScribe and Atomic Simulation Environment (ASE)
# 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from dscribe.descriptors import CoulombMatrix
from ase.build import molecule
from ase.io import read
# This code block shows the default parameters for generating CMs 
# in DScribe. By default, a sorted CM is generate with L2-norm 
# sorted rows and columns. n_atoms_max is a parameter that helps
# with padding, for one molecule it is the number of atoms in 
# the molecule.
print('Sorted CM')
cm = CoulombMatrix(n_atoms_max=11)

# Generate the ethanol molecule using ASE

mol = read(path_to_structure)
print(type(mol))

# Create CM output for the system
cm_mol = cm.create(mol)

# Print the sorted CM and it's corresponding shape
print("Flattened shape of the sorted CM", cm_mol.shape)
print(cm_mol)

# Set the parameters, sigma and seed, of the unsorted CM
print('\nUnsorted CM')
cm_unsrtd=CoulombMatrix(n_atoms_max=11,permutation='none')

# Generate the unsorted CM for mol
cm_unsrtd_mol = cm_unsrtd.create(mol)

# Print the unsorted CM and it's corresponding shape
print("Flattened shape of the unsorted CM", cm_unsrtd_mol.shape)
print(cm_unsrtd_mol)

# Set the parameters of the eigenspectrum representation
print('\nEigenspectrum')
cm_eigen=CoulombMatrix(n_atoms_max=11,permutation='eigenspectrum')

# Generate the eigenspectrum representation of the CM
cm_eigen_mol = cm_eigen.create(mol)

# Print the eigenspectrum and it's corresponding shape
print("Flattened shape of the eigenspectrum", cm_eigen_mol.shape)
print(cm_eigen_mol)

# Set the parameters, sigma and seed, of the randomly sorted CM
# Examine how sigma effects the sorting of the randomly sorted CM
print('\nRandomly sorted CM')
cm_random=CoulombMatrix(n_atoms_max=11,permutation='random',
                        sigma=1e-3,seed=42)

# Generate the randomly sorted CM for mol
cm_random_mol = cm_random.create(mol)

# Print the randomly sorted CM and it's corresponding shape
print("Flattened shape of the randomly sorted CM", 
      cm_random_mol.shape)
print(cm_random_mol)

In [ ]:
# Generate the unsorted CM of glycidol using a seaborn heatmap
print(mol.get_chemical_symbols())
print("Double check ordering", mol)
ds_CMdf=pd.DataFrame(cm_unsrtd_mol.reshape(11,11),columns=mol.get_chemical_symbols(),index=mol.get_chemical_symbols())
srted_ds_CMdf=ds_CMdf.loc[['O', 'C','H'],['O', 'C','H']]

In [ ]:
srted_ds_CMdf

In [ ]:
cm_1 = srted_ds_CMdf.to_numpy().reshape(11,11)
row_norm = np.array(sorted([(np.linalg.norm(i),idx) for idx,i in enumerate(cm_1)],reverse=True))[:,1].astype(int)
column_norm = np.array(sorted([(np.linalg.norm(i),idx) for idx,i in enumerate(cm_1[row_norm].T)],reverse=True))[:,1].astype(int)
mysrt=cm_1[row_norm][column_norm]

In [ ]:
mysrt == cm_mol.reshape(11,11)

In [ ]:
srted_ds_CMdf.to_numpy()==mysrt

In [ ]:
plt.figure(figsize=(5,5),dpi=300)
sns.heatmap(srted_ds_CMdf,cmap=sns.cm.rocket_r,cbar=False,square=True,annot=True, fmt='.2f',linewidths=0.1, annot_kws={'fontsize':8})
plt.tight_layout()
plt.savefig('glycidol_CM.png',dpi=300)
plt.savefig('glycidol_CM.svg',dpi=300)
plt.show()

In [ ]:
# Getting the Upper Triangle of the co-relation matrix



plt.figure(figsize=(5,5),dpi=300)

# Create a mask
mask = np.tril(np.ones_like(srted_ds_CMdf, dtype=bool))
np.fill_diagonal(mask, True)


sns.heatmap(srted_ds_CMdf, mask=mask,cmap=sns.cm.rocket_r,cbar=False,square=True,annot=True, fmt='.2f',linewidths=0.1, annot_kws={'fontsize':8})
plt.tight_layout()

In [ ]:
(len(mol.get_atomic_numbers())*(len(mol.get_atomic_numbers())+1))/2

In [ ]:
cm_unsrtd_mol.reshape(11,11)

In [ ]:
idx = 0
for idi, Zi in enumerate(mol.get_atomic_numbers()):
    for idj, Zj in enumerate(mol.get_atomic_numbers()):
        if idi != idj:
            dist = mol.get_all_distances()
            my_cm = (Zi*Zj)/dist[idi,idj]
            if Zi==6 and Zj==8:
                print(my_cm,dist[idi,idj])
        else:
            my_cm = 0.5 * Zi**2.4
        # print(my_cm==cm_unsrtd_mol[idx])
        idx+=1